# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 26 2022 1:39PM,247292,10,SONSB0001953,"Nextsource Biotechnology, LLC",Released
1,Sep 26 2022 1:39PM,247292,10,SONSB0001954,"Nextsource Biotechnology, LLC",Released
2,Sep 26 2022 1:31PM,247290,12,8576145,LF of America Corp.,Released
3,Sep 26 2022 1:21PM,247288,16,8565074,Sartorius Lab Products and Service,Released
4,Sep 26 2022 1:19PM,247282,10,CLI00389.1,"CLINUVEL, Inc.",Released
5,Sep 26 2022 1:12PM,247283,10,EYE42991,Eye Pharma Inc,Released
6,Sep 26 2022 1:12PM,247283,10,EYE42992,Eye Pharma Inc,Released
7,Sep 26 2022 1:11PM,247287,19,ADV80005911,"AdvaGen Pharma, Ltd",Released
8,Sep 26 2022 1:10PM,247286,16,8560865,Sartorius Bioprocess Solutions,Released
9,Sep 26 2022 1:10PM,247286,16,8564511,Sartorius Bioprocess Solutions,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,247286,Released,3
31,247287,Released,1
32,247288,Released,1
33,247290,Released,1
34,247292,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247286,NaN,NaN,3.0
247287,NaN,NaN,1.0
247288,NaN,NaN,1.0
247290,NaN,NaN,1.0
247292,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247197,0.0,1.0,0.0
247219,0.0,8.0,7.0
247220,0.0,0.0,3.0
247223,0.0,0.0,8.0
247238,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247197,0,1,0
247219,0,8,7
247220,0,0,3
247223,0,0,8
247238,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247197,0,1,0
1,247219,0,8,7
2,247220,0,0,3
3,247223,0,0,8
4,247238,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247197,,1,
1,247219,,8,7
2,247220,,,3
3,247223,,,8
4,247238,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 26 2022 1:39PM,247292,10,"Nextsource Biotechnology, LLC"
2,Sep 26 2022 1:31PM,247290,12,LF of America Corp.
3,Sep 26 2022 1:21PM,247288,16,Sartorius Lab Products and Service
4,Sep 26 2022 1:19PM,247282,10,"CLINUVEL, Inc."
5,Sep 26 2022 1:12PM,247283,10,Eye Pharma Inc
7,Sep 26 2022 1:11PM,247287,19,"AdvaGen Pharma, Ltd"
8,Sep 26 2022 1:10PM,247286,16,Sartorius Bioprocess Solutions
11,Sep 26 2022 1:07PM,247285,10,Emerginnova
14,Sep 26 2022 1:00PM,247284,10,"Senseonics, Incorporated"
16,Sep 26 2022 12:55PM,247281,19,"NAPP Technologies, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 26 2022 1:39PM,247292,10,"Nextsource Biotechnology, LLC",,,2
1,Sep 26 2022 1:31PM,247290,12,LF of America Corp.,,,1
2,Sep 26 2022 1:21PM,247288,16,Sartorius Lab Products and Service,,,1
3,Sep 26 2022 1:19PM,247282,10,"CLINUVEL, Inc.",,,1
4,Sep 26 2022 1:12PM,247283,10,Eye Pharma Inc,,,2
5,Sep 26 2022 1:11PM,247287,19,"AdvaGen Pharma, Ltd",,,1
6,Sep 26 2022 1:10PM,247286,16,Sartorius Bioprocess Solutions,,,3
7,Sep 26 2022 1:07PM,247285,10,Emerginnova,,,3
8,Sep 26 2022 1:00PM,247284,10,"Senseonics, Incorporated",,,2
9,Sep 26 2022 12:55PM,247281,19,"NAPP Technologies, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 26 2022 1:39PM,247292,10,"Nextsource Biotechnology, LLC",2,,
1,Sep 26 2022 1:31PM,247290,12,LF of America Corp.,1,,
2,Sep 26 2022 1:21PM,247288,16,Sartorius Lab Products and Service,1,,
3,Sep 26 2022 1:19PM,247282,10,"CLINUVEL, Inc.",1,,
4,Sep 26 2022 1:12PM,247283,10,Eye Pharma Inc,2,,
5,Sep 26 2022 1:11PM,247287,19,"AdvaGen Pharma, Ltd",1,,
6,Sep 26 2022 1:10PM,247286,16,Sartorius Bioprocess Solutions,3,,
7,Sep 26 2022 1:07PM,247285,10,Emerginnova,3,,
8,Sep 26 2022 1:00PM,247284,10,"Senseonics, Incorporated",2,,
9,Sep 26 2022 12:55PM,247281,19,"NAPP Technologies, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 26 2022 1:39PM,247292,10,"Nextsource Biotechnology, LLC",2,,
1,Sep 26 2022 1:31PM,247290,12,LF of America Corp.,1,,
2,Sep 26 2022 1:21PM,247288,16,Sartorius Lab Products and Service,1,,
3,Sep 26 2022 1:19PM,247282,10,"CLINUVEL, Inc.",1,,
4,Sep 26 2022 1:12PM,247283,10,Eye Pharma Inc,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 26 2022 1:39PM,247292,10,"Nextsource Biotechnology, LLC",2.0,NaN,NaN
1,Sep 26 2022 1:31PM,247290,12,LF of America Corp.,1.0,NaN,NaN
2,Sep 26 2022 1:21PM,247288,16,Sartorius Lab Products and Service,1.0,NaN,NaN
3,Sep 26 2022 1:19PM,247282,10,"CLINUVEL, Inc.",1.0,NaN,NaN
4,Sep 26 2022 1:12PM,247283,10,Eye Pharma Inc,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 26 2022 1:39PM,247292,10,"Nextsource Biotechnology, LLC",2.0,0.0,0.0
1,Sep 26 2022 1:31PM,247290,12,LF of America Corp.,1.0,0.0,0.0
2,Sep 26 2022 1:21PM,247288,16,Sartorius Lab Products and Service,1.0,0.0,0.0
3,Sep 26 2022 1:19PM,247282,10,"CLINUVEL, Inc.",1.0,0.0,0.0
4,Sep 26 2022 1:12PM,247283,10,Eye Pharma Inc,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4945272,49.0,10.0,2.0
12,247290,1.0,0.0,0.0
16,989075,6.0,0.0,0.0
19,741826,3.0,0.0,0.0
20,247262,46.0,5.0,0.0
21,494443,0.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4945272,49.0,10.0,2.0
1,12,247290,1.0,0.0,0.0
2,16,989075,6.0,0.0,0.0
3,19,741826,3.0,0.0,0.0
4,20,247262,46.0,5.0,0.0
5,21,494443,0.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,49.0,10.0,2.0
1,12,1.0,0.0,0.0
2,16,6.0,0.0,0.0
3,19,3.0,0.0,0.0
4,20,46.0,5.0,0.0
5,21,0.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,49.0
1,12,Released,1.0
2,16,Released,6.0
3,19,Released,3.0
4,20,Released,46.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Completed,2.0,0.0,0.0,0.0,0.0,0.0
Executing,10.0,0.0,0.0,0.0,5.0,2.0
Released,49.0,1.0,6.0,3.0,46.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,10.0,0.0,0.0,0.0,5.0,2.0
2,Released,49.0,1.0,6.0,3.0,46.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,10.0,0.0,0.0,0.0,5.0,2.0
2,Released,49.0,1.0,6.0,3.0,46.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()